In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as image
import math

### Smoothening(Noise Reduction)
Smoothing is  used to reduce noise within an image or to produce a less pixelated image. Most smoothing methods are based on low pass filters.  Smoothing is also usually based on a single value representing the image, such as the average value of the image or the middle (median) value. There are alsso blur techniues such as Gaussian Blur which basically involves blurring an image through a Gaussian Function.

In [2]:
def smooth(image, threshold):
    height = image.shape[0]
    width = image.shape[1]
    for i in range(height-1):
        for j in range(width-1):
            if image[i, j] < threshold:
                image[i, j] = 255
            else:
                image[i, j] = 0
    cv.imshow('my_blur', image)
    return image

### Image thresholding
Image thresholding is a  way of partitioning an image into a foreground and background. This image analysis technique is a type of image segmentation that isolates objects by converting grayscale images into binary images.

Original image
<img src="https://upload.wikimedia.org/wikipedia/commons/d/d6/Pavlovsk_Railing_of_bridge_Yellow_palace_Winter.jpg">

Binary image

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d4/Pavlovsk_Railing_of_bridge_Yellow_palace_Winter_bw_threshold.jpg/1024px-Pavlovsk_Railing_of_bridge_Yellow_palace_Winter_bw_threshold.jpg">

 ## Thresholding

#### Simple Thresholding
For every pixel, the same threshold value is applied. If the pixel value is smaller than the threshold, it is set to 0, otherwise it is set to a maximum value. 

#### OTSU METHOD of thresholding
Otsu's thresholding method involves iterating through all the possible threshold values and calculating a measure of spread for the pixel levels each side of the threshold, i.e. the pixels that either fall in foreground or background. The aim is to find the threshold value where the sum of foreground and background spreads is at its minimum.

#### Adaptive Thresholding
Here, the algorithm determines the threshold for a pixel based on a small region around it. So we get different thresholds for different regions of the same image which gives better results for images with varying illumination.


In [3]:
def threshold_op(gray, mode):
    if mode == 0:  
          ret, binary = cv.threshold(gray, 0, 255,   cv.THRESH_BINARY | cv.THRESH_OTSU) 
    if mode==1:
        ret, binary = cv.threshold(gray, 180, 255,cv.THRESH_BINARY) # Simple Thresholding 
    if mode == 2:  
        binary = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C,
                                      cv.THRESH_BINARY,15,25)
    cv.imshow("threshold", binary)
    return binary

## Morphological Operations
Morphological operations are a set of operations that process image based on their shapes. They apply a structuring element to the input image and get an output image. 

### Structural Element
A structuring element is a matrix that identifies the pixel in the image being processed and defines the neighborhood used in the processing of each pixel.


In morphological filter, each element in the matrix is called “structuring element” instead of coefficient matrix in the linear filter. The structuring elements contain only the value 0 and 1. And the hot spot of the filter is the dark shade element.

<img src="https://cdn-images-1.medium.com/max/1600/1*Bv0axRuLXhDH0vL_U5ApCA.png">

The binary image is described as sets of two-dimensional coordinate point. This is called “Point Set” Q and point set consist of the coordinate pair p = (u,v) of all foreground pixels. Some operations of point set are similar to the operation in others image. For inverting binary image is complement operation and combining two binary image use union operator. Shifting binary image I by some coordinate vector d by adding vector d to point p. Or reflection of binary image I by multiply -1 to point p.

In gray-scale morphology, structuring elements are defined as real-value 2D functions H:$\mathbb{Z}^2\rightarrow \mathbb{R}$  instead of point sets. 

<img src="https://cdn-images-1.medium.com/max/1400/1*u7i_BHDbE9d8f0PHfETb4g.png">

The value in H can be negative or zero value. But it contrast to linear convolution, zero elements are used to compute the result. And if do not want to use the elements in some location, one can put no element in that location.

<img src="https://cdn-images-1.medium.com/max/1600/1*OfMAeRHr5qDtMRJ9SQLQrA.png">



### Dilation
Dilation is a morphological operator which work for the grow process as I mentioned before. The equation of this operator is defined as

<img src="https://cdn-images-1.medium.com/max/1600/1*kHnFPc3pHCvefc77f48vCw.png">

It consists of convoluting an image I with some kernel B, of any shape or size, usually a square or a circle. The kernel B has a defined anchor point [The anchor point defines how the kernel is positioned with respect to the pixel currently processed during the filter operation.]. As the kernel B is scanned over the image, we compute the maximal pixel value overlapped by B and replace the image pixel in the anchor point position with that maximal value. As you can deduce, this maximizing operation causes bright regions within an image to “grow” (therefore the name dilation).

In gray scale morphology, it is the maximum value of the value in H add to the current sub-image.

<img src="https://cdn-images-1.medium.com/max/1200/1*jDswdRMsFAnBM85vMiY78g.png">


### Erosion
Erosion is a morphological operator which work for the shrink process as I mentioned before as well and the equation is defined as

<img src="https://cdn-images-1.medium.com/max/1000/1*k22ghIKh6BCQIcB64zc6ww.png">

Unlike dilation, it compute a local minimum over the area of the kernel.As the kernel B is scanned over the image, we compute the minimal pixel value overlapped by B and replace the image pixel under the anchor point with that minimal value. 

In gray scale morphology, it is the minimum value of the difference.
<img src="https://cdn-images-1.medium.com/max/1200/1*at5Ax1pFvjGdzv3enFWrSg.png">

#### Note
In addition, erosion and dilation are duels, for a dilation of the foreground can be accomplished by an erosion of background and subsequent of the result in two different properties but work similarily.

<img src="https://cdn-images-1.medium.com/max/1400/1*6gs1qFa5j_Sp8LYGGnNJ-w.png">


### Opening
It is  the erosion of an image followed by a dilation(dilate(erode(src,element)).It is useful for removing small objects (it is assumed that the objects are bright on a dark foreground)

<img src="https://opencv-python-tutroals.readthedocs.io/en/latest/_images/opening.png">

### Closing
It is  the dilation of an image followed by a erosion(erode(dilation(src,element)).It is useful to remove small holes (dark regions).

<img src="https://opencv-python-tutroals.readthedocs.io/en/latest/_images/closing.png">

In [ ]:
def mor_op(binary, mode, iterations):
    if mode == 'close':
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))  
        sure = cv.morphologyEx(binary, cv.MORPH_CLOSE, kernel, iterations=iterations)  
        morimage = cv.bitwise_not(sure) 
        morimage = cv.bilateralFilter(morimage, 0, 100, 15)
        morimage=smooth(morimage, 155)
        cv.imshow('morimage', morimage)
    elif mode == 'open':
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))  
        morimage = cv.morphologyEx(binary, cv.MORPH_OPEN, kernel, iterations) 
        morimage=smooth(morimage, 200)
        cv.imshow('mor_op', morimage)
    return morimage

In [ ]:
image = cv.imread('tile_582.jpg')  
#image = cv.GaussianBlur(image, (3, 3), 0) ### It is used for noise reduction in an image(Alternative for manual smoothening function.)
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)  
gray=cv.equalizeHist(gray) ' It is a method in image processing of contrast adjustment using the image histogram.'
cv.imshow('equalHist', gray)  
binary = threshold_op(gray, 2)  
morimage = mor_op(binary,'open',20) 'For defect detection in Ceramic tiles , opening works better than closing.'
cv.waitKey(0)


## Comment
The algorithms that are adapted in the paper https://moodle.cmi.ac.in/pluginfile.php/10112/mod_resource/content/2/ceramic-tiles.pdf for surface defect detection in Ceramic tiles are basically combination of various morphological operations ,thresholding, smoothening techniques. The paper though succeeds in idefect detection quite often but does not provide a walk through approach and thus, requires tuning of the parameters involved in its implementation. 